# Análise Detalhada dos Resultados - Pipeline CNN AV1 (Bloco 16x16)

Este notebook apresenta uma análise completa dos resultados do pipeline hierárquico de predição de particionamento AV1 para blocos de tamanho 16x16.

## Estrutura da Análise

1. **Carregamento e Exploração dos Dados**
2. **Desempenho Geral do Pipeline**
3. **Stage 1: Classificador Binário**
4. **Stage 2: Macro Classes**
5. **Stage 3: Especialistas**
6. **Matriz de Confusão Final**
7. **Métricas por Classe**
8. **Análise de Erros**
9. **Impacto do Threshold Stage 1**
10. **Visualizações Consolidadas**

## 1. Carregamento e Exploração dos Dados

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path

# Configurar estilo dos gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configurar tamanho padrão das figuras
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("Bibliotecas importadas com sucesso!")

In [ ]:
# Carregar dados JSON
json_path = Path('../logs/v5_pipeline_eval_val_th050.json')
csv_path = Path('../logs/v5_pipeline_eval_val_th050.csv')

with open(json_path, 'r') as f:
    results = json.load(f)

# Carregar CSV com predições detalhadas
df_predictions = pd.read_csv(csv_path)

# Exibir informações básicas
print(f"✓ JSON carregado: {json_path}")
print(f"✓ CSV carregado: {csv_path}")
print(f"\nTotal de amostras: {len(df_predictions)}")
print(f"Threshold Stage1: {results['stage1_threshold']}")
print(f"Dataset: {results['dataset']}")
print(f"\nEspecialistas disponíveis: {', '.join(results['available_specialists'])}")
print(f"\n{'='*60}")
print(f"ACURÁCIA FINAL DO PIPELINE: {results['final_accuracy']*100:.2f}%")
print(f"{'='*60}")

In [ ]:
# Explorar estrutura do DataFrame
print("Colunas do CSV:")
print(df_predictions.columns.tolist())
print(f"\nPrimeiras linhas:")
df_predictions.head(10)

## 2. Desempenho Geral do Pipeline

In [ ]:
# Calcular distribuição das classes reais
class_distribution = df_predictions['stage0_true'].value_counts().sort_index()
total_samples = len(df_predictions)

# Visualizar distribuição
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
axes[0].bar(range(len(class_distribution)), class_distribution.values, color='steelblue')
axes[0].set_xticks(range(len(class_distribution)))
axes[0].set_xticklabels(class_distribution.index, rotation=45, ha='right')
axes[0].set_title('Distribuição das Classes Reais (Ground Truth)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Número de Amostras')
axes[0].grid(axis='y', alpha=0.3)

# Adicionar valores sobre as barras
for i, v in enumerate(class_distribution.values):
    axes[0].text(i, v + 100, f'{v}\n({v/total_samples*100:.1f}%)', ha='center', va='bottom', fontsize=9)

# Gráfico de pizza (apenas top classes)
top_classes = class_distribution.head(5)
others = class_distribution[5:].sum()
pie_data = list(top_classes.values) + [others]
pie_labels = list(top_classes.index) + ['Outros']
colors = plt.cm.Set3(range(len(pie_data)))

axes[1].pie(pie_data, labels=pie_labels, autopct='%1.1f%%', colors=colors, startangle=90)
axes[1].set_title('Proporção das 5 Classes Principais', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n{'='*70}")
print(f"DISTRIBUIÇÃO DAS CLASSES:")
print(f"{'='*70}")
for cls, count in class_distribution.items():
    print(f"{cls:20s}: {count:6d} ({count/total_samples*100:5.2f}%)")

In [ ]:
# Calcular acurácia geral e por classe
correct_predictions = (df_predictions['stage0_true'] == df_predictions['final_pred']).sum()
overall_accuracy = correct_predictions / total_samples

# Acurácia por classe
per_class_accuracy = {}
for cls in class_distribution.index:
    mask = df_predictions['stage0_true'] == cls
    if mask.sum() > 0:
        correct = ((df_predictions['stage0_true'] == df_predictions['final_pred']) & mask).sum()
        per_class_accuracy[cls] = correct / mask.sum()
    else:
        per_class_accuracy[cls] = 0.0

# Visualizar
fig, ax = plt.subplots(figsize=(14, 6))
classes = list(per_class_accuracy.keys())
accuracies = list(per_class_accuracy.values())

bars = ax.bar(range(len(classes)), accuracies, color='coral')
ax.axhline(y=overall_accuracy, color='red', linestyle='--', linewidth=2, label=f'Acurácia Geral: {overall_accuracy*100:.2f}%')
ax.set_xticks(range(len(classes)))
ax.set_xticklabels(classes, rotation=45, ha='right')
ax.set_ylabel('Acurácia')
ax.set_title('Acurácia por Classe de Partição', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 1.0)

# Adicionar valores sobre as barras
for i, v in enumerate(accuracies):
    ax.text(i, v + 0.02, f'{v*100:.1f}%', ha='center', va='bottom', fontsize=9)

# Destacar classes com acurácia < 20% em vermelho
for i, v in enumerate(accuracies):
    if v < 0.2:
        bars[i].set_color('darkred')

plt.tight_layout()
plt.show()

print(f"\n{'='*70}")
print(f"ACURÁCIA POR CLASSE:")
print(f"{'='*70}")
for cls, acc in sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True):
    status = "⚠️ CRÍTICO" if acc < 0.2 else "✓" if acc > 0.5 else "⚠️"
    print(f"{cls:20s}: {acc*100:6.2f}% {status}")

## 3. Stage 1: Classificador Binário (Particiona vs Não-Particiona)

In [ ]:
# Extrair métricas do Stage 1
s1 = results['stage1']

print(f"{'='*70}")
print(f"STAGE 1 - CLASSIFICADOR BINÁRIO")
print(f"{'='*70}")
print(f"Acurácia:  {s1['accuracy']*100:.2f}%")
print(f"Precisão:  {s1['precision']*100:.2f}%")
print(f"Recall:    {s1['recall']*100:.2f}%")
print(f"F1-Score:  {s1['f1']*100:.2f}%")
print(f"\nConfusão:")
print(f"  True Positives  (TP): {s1['tp']:,}")
print(f"  False Positives (FP): {s1['fp']:,}")
print(f"  False Negatives (FN): {s1['fn']:,}")
print(f"\nInterpretação:")
print(f"  - Alto recall ({s1['recall']*100:.1f}%) = captura {s1['recall']*100:.1f}% dos blocos que devem particionar")
print(f"  - Baixa precisão ({s1['precision']*100:.1f}%) = ~{(1-s1['precision'])*100:.1f}% de falsos positivos")
print(f"  - {s1['fp']:,} blocos NONE enviados incorretamente para Stage 2")

# Visualizar matriz de confusão Stage 1
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Matriz de confusão
tn = len(df_predictions) - s1['tp'] - s1['fp'] - s1['fn']  # True Negatives
cm_s1 = np.array([[tn, s1['fp']], 
                  [s1['fn'], s1['tp']]])

sns.heatmap(cm_s1, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Pred: NONE', 'Pred: Particiona'],
            yticklabels=['Real: NONE', 'Real: Particiona'])
axes[0].set_title('Matriz de Confusão - Stage 1', fontsize=14, fontweight='bold')

# Métricas comparativas
metrics = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']
values = [s1['accuracy'], s1['precision'], s1['recall'], s1['f1']]
colors_met = ['#2ecc71' if v > 0.7 else '#e74c3c' if v < 0.6 else '#f39c12' for v in values]

bars = axes[1].barh(metrics, values, color=colors_met)
axes[1].set_xlim(0, 1.0)
axes[1].set_xlabel('Valor')
axes[1].set_title('Métricas do Stage 1', fontsize=14, fontweight='bold')
axes[1].axvline(x=0.7, color='green', linestyle='--', alpha=0.5, label='Bom (>70%)')
axes[1].axvline(x=0.6, color='orange', linestyle='--', alpha=0.5, label='Moderado (>60%)')
axes[1].legend()

for i, v in enumerate(values):
    axes[1].text(v + 0.02, i, f'{v*100:.1f}%', va='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

## 4. Stage 2: Classificação em Macro Classes

In [ ]:
# Extrair dados do Stage 2
s2 = results['stage2']
cm_s2 = np.array(s2['confusion_matrix'])
class_names_s2 = s2['class_names']

print(f"{'='*70}")
print(f"STAGE 2 - MACRO CLASSES")
print(f"{'='*70}")
print(f"Macro F1-Score: {s2['macro_f1']*100:.2f}%")
print(f"\nClasses: {', '.join(class_names_s2)}")

# Visualizar matriz de confusão Stage 2
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Heatmap da matriz de confusão
sns.heatmap(cm_s2, annot=True, fmt='d', cmap='YlOrRd', ax=axes[0],
            xticklabels=class_names_s2, yticklabels=class_names_s2)
axes[0].set_xlabel('Predito')
axes[0].set_ylabel('Real')
axes[0].set_title('Matriz de Confusão - Stage 2', fontsize=14, fontweight='bold')

# Calcular métricas por classe
from sklearn.metrics import precision_recall_fscore_support

# Filtrar apenas linhas/colunas com dados (excluir NONE e 1TO4)
mask = (cm_s2.sum(axis=1) > 0) & (cm_s2.sum(axis=0) > 0)
cm_s2_filtered = cm_s2[mask][:, mask]
active_classes = [class_names_s2[i] for i in range(len(class_names_s2)) if mask[i]]

# Calcular métricas
y_true = []
y_pred = []
for i in range(len(cm_s2_filtered)):
    for j in range(len(cm_s2_filtered[i])):
        y_true.extend([i] * cm_s2_filtered[i][j])
        y_pred.extend([j] * cm_s2_filtered[i][j])

precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average=None)

# Gráfico de barras com métricas
x = np.arange(len(active_classes))
width = 0.25

bars1 = axes[1].bar(x - width, precision, width, label='Precisão', color='skyblue')
bars2 = axes[1].bar(x, recall, width, label='Recall', color='lightcoral')
bars3 = axes[1].bar(x + width, f1, width, label='F1-Score', color='lightgreen')

axes[1].set_xlabel('Macro Classes')
axes[1].set_ylabel('Score')
axes[1].set_title('Métricas por Macro Classe - Stage 2', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(active_classes, rotation=45, ha='right')
axes[1].legend()
axes[1].set_ylim(0, 1.0)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Exibir métricas detalhadas
print(f"\n{'='*70}")
print(f"MÉTRICAS POR MACRO CLASSE:")
print(f"{'='*70}")
print(f"{'Classe':<15} {'Support':>10} {'Precisão':>12} {'Recall':>12} {'F1-Score':>12}")
print(f"{'-'*70}")
for i, cls in enumerate(active_classes):
    print(f"{cls:<15} {support[i]:>10} {precision[i]*100:>11.2f}% {recall[i]*100:>11.2f}% {f1[i]*100:>11.2f}%")